## Mace MD Demo notebook

This should demonstrate the bare bones of the API for setting up a variety of MD jobs with hybrid MACE-classical hamiltonians.  The same functionalty is exposed through the `mace-md` command line program that is installed with the `mace-openmm-interop` package


Your vs code server instance should have an attached CUDA GPU for this to run!

In [1]:
!nvidia-smi

Tue Feb 21 15:56:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:C1:00.0 Off |                    0 |
| N/A   35C    P0    79W / 500W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from openmmtools.openmm_torch.hybrid_md import PureSystem, MixedSystem
import torch
import logging
from typing import Optional, Union
from mace import tools

/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:89: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  [path_part.strip("/ ") for path_part in split_path if path_part is not ""]
/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:729: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._output_mode is "a":


In [10]:
tools.setup_logger(level="INFO", directory="./logs")

In [5]:
# Change inputs in this cell only
file = "./example_data/ejm_31.sdf"  # path to the system to be simulated, can be either protein ligand complex or isolated (sdf)
ml_mol = "./example_data/ejm_31.sdf"  # path to the proper topology of the small molecule to be modeleld by mace, since pdb files are not sufficiently detailed.  this should be sdf.
model_path = "./example_data/MACE_SPICE_larger.model"
forcefields = [
    "amber/protein.ff14SB.xml",
    "amber/tip3p_standard.xml",
    "amber14/DNA.OL15.xml",
]
# name of the residue to be parametrised by mace, should correspond to whatever it is called in the ml_mol sdf file and system file
resname = "4GIH_lig_ejm_31"
# minimum separation between the solute and box wall
padding = 1.2
# ion concentration in the solute
ionicStrength = 0.15
# cutoff distance (nm) for the non-bonded terns of the classical forcefield
nonbondedCutoff = 1.0
# name of the neural network potential, should correpond to the implemented FFs
# in the openmm-ml package
potential = "mace"
# simulation temperature
temperature = 298.15
# precision to use for both MACE and openMM
dtype = torch.float64
# which version of the torch_nl to use - the n^2 impl is more reliable
neighbour_list = "torch_nl_n2"
# directory where simulation output will be written
output_dir = "./output"
# specify the type of system to create - pure (just the solute simulated in vcuum)
# hybrid (small molecule modelled by MACE, rest of the system (protein or solvent) modelled by classical forcefield), decoupled (for ABFE simulations - lambda parameter controls switching off the ligand non-bonded terms)

In [6]:
torch.set_default_dtype(dtype)

## Run pure MD of the small molecule in vacuum - no periodic boundary conditions

In [7]:
system=PureSystem(
    file=file,
    model_path=model_path,
    potential="mace",
    temperature=temperature,
    output_dir="output_pure_mace"
)

2023-02-21 15:57:02.939 DEBUG: OpenMM will use Double precision
2023-02-21 15:57:02.939 INFO: Using SMFF openff_unconstrained-1.0.0.offxml
2023-02-21 15:57:02.940 DEBUG: OpenMM will use Double precision
2023-02-21 15:57:02.956 INFO: Initialized topology with (32, 3) positions
MACE model compiled
False
rmax from model is 4.5


In [8]:
system.run_mixed_md(
    steps=1000, interval=100, output_file="output_md_test.pdb"
)

2023-02-21 15:51:10.254 DEBUG: Running mixed MD for 1000 steps
2023-02-21 15:51:11.315 INFO: Minimising energy...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-309.32879977405827,34.20293670313781,0
200,0.20000000000000015,-298.2740657364288,61.53124411321979,2.5
300,0.3000000000000002,-287.2109490222236,96.96960603084678,2.51
400,0.4000000000000003,-273.9624983082774,118.70448190620105,2.52
500,0.5000000000000003,-276.8225939343829,126.58237166354131,2.52
600,0.6000000000000004,-262.94036164742255,140.83062003311687,2.52
700,0.7000000000000005,-258.69811508348124,183.64048520713902,2.52
800,0.8000000000000006,-243.63524872479533,163.21396965787534,2.52
900,0.9000000000000007,-227.26828931705077,187.22719605792372,2.52
1000,1.0000000000000007,-231.83452936886613,201.1611962736112,2.52


## Run Mixed MD with the ligand solvated in a tip3p water box

Construct a system where the bonded terms for the small molecule are replaced by MACE, solvent and long-range interactions retain the AMBER FF parameters

Small molecule in periodic water box

In [8]:
mixedSystem = MixedSystem(
    file=file,
    ml_mol=ml_mol,
    model_path=model_path,
    forcefields=forcefields,
    resname=resname,
    nnpify_type="resname",
    padding=padding,
    ionicStrength=ionicStrength,
    nonbondedCutoff=nonbondedCutoff,
    potential=potential,
    temperature=temperature,
    dtype=dtype,
    neighbour_list=neighbour_list,
    output_dir=output_dir,
)

2023-02-21 16:03:08.921 DEBUG: OpenMM will use Double precision
2023-02-21 16:03:08.923 INFO: Using SMFF openff_unconstrained-1.0.0.offxml
2023-02-21 16:03:08.924 DEBUG: OpenMM will use Double precision
2023-02-21 16:03:08.955 INFO: Initialized topology with (32, 3) positions
2023-02-21 16:03:09.343 INFO: Requested to generate parameters for residue <Residue 0 (4GIH_lig_ejm_31) of chain 0>
2023-02-21 16:03:09.555 INFO: Generating a residue template for [H][c]1[n][c]([N]([H])[C](=[O])[C]([H])([H])[H])[c]([H])[c]([N]([H])[C](=[O])[c]2[c]([Cl])[c]([H])[c]([H])[c]([H])[c]2[Cl])[c]1[H] using openff_unconstrained-1.0.0.offxml
2023-02-21 16:03:09.986 DEBUG: Changing app registry from <pint.registry.LazyRegistry object at 0x1528dcf9cd30> to <pint.registry.UnitRegistry object at 0x152883a9b970>.
2023-02-21 16:03:09.988 DEBUG: Generating parameters...
2023-02-21 16:03:09.993 DEBUG: Finding matches for ProperTorsionHandler
2023-02-21 16:03:09.995 DEBUG: [*:1]-[#6X4:2]-[#6X4:3]-[*:4]              

2023-02-21 16:03:10.080 DEBUG: [#6X3:1]=[#7X2,#7X3+1:2]-[#6X4:3]-[#1:4]                         :        0 matches
2023-02-21 16:03:10.081 DEBUG: [#6X3:1]=[#7X2,#7X3+1:2]-[#6X4:3]-[#6X3,#6X4:4]                  :        0 matches
2023-02-21 16:03:10.083 DEBUG: [*:1]~[#7X3,#7X2-1:2]-[#6X3:3]~[*:4]                             :       16 matches
2023-02-21 16:03:10.084 DEBUG: [*:1]~[#7X3,#7X2-1:2]-!@[#6X3:3]~[*:4]                           :       16 matches
2023-02-21 16:03:10.086 DEBUG: [#1:1]-[#7X3:2]-[#6X3:3]=[#8,#16,#7:4]                           :        2 matches
2023-02-21 16:03:10.087 DEBUG: [*:1]-[#7X3;r5:2]-@[#6X3;r5:3]~[*:4]                             :        0 matches
2023-02-21 16:03:10.088 DEBUG: [#8X1:1]~[#7X3:2]~[#6X3:3]~[*:4]                                 :        0 matches
2023-02-21 16:03:10.090 DEBUG: [*:1]=[#7X2,#7X3+1:2]-[#6X3:3]-[*:4]                             :        0 matches
2023-02-21 16:03:10.091 DEBUG: [*:1]=[#7X2,#7X3+1:2]-[#6X3:3]=,:[*:4]           

2023-02-21 16:03:10.175 DEBUG: [#6X3:1]-[#16X4,#16X3+0:2]-[#7X4,#7X3:3]-[#6X4:4]                :        0 matches
2023-02-21 16:03:10.176 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X4,#7X3:3]-[#1:4]                  :        0 matches
2023-02-21 16:03:10.178 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X4,#7X3:3]-[#6X4:4]                :        0 matches
2023-02-21 16:03:10.179 DEBUG: [#6X3:1]-[#16X4,#16X3+0:2]-[#7X3:3]-[#6X3:4]                     :        0 matches
2023-02-21 16:03:10.180 DEBUG: [#6X4:1]-[#16X4,#16X3+0:2]-[#7X3:3]-[#6X3:4]                     :        0 matches
2023-02-21 16:03:10.182 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X3:3]-[#6X3:4]                     :        0 matches
2023-02-21 16:03:10.183 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X3:3]-[#7X2:4]                     :        0 matches
2023-02-21 16:03:10.184 DEBUG: [*:1]~[#16X4,#16X3+0:2]=,:[#7X2:3]-,:[*:4]                       :        0 matches
2023-02-21 16:03:10.186 DEBUG: [#6X4:1]-[#16X4,#16X3+0:2]-[#7X2:3]~[#6X3:4]     

2023-02-21 16:03:10.270 DEBUG: [#6X3:1]-[#7X3:2]                                                :        4 matches
2023-02-21 16:03:10.271 DEBUG: [#6X4:1]-[#7X3:2]-[#6X3]=[#8X1+0]                                :        0 matches
2023-02-21 16:03:10.273 DEBUG: [#6X3:1](=[#8X1+0])-[#7X3:2]                                     :        2 matches
2023-02-21 16:03:10.274 DEBUG: [#6X3:1]-[#7X2:2]                                                :        0 matches
2023-02-21 16:03:10.275 DEBUG: [#6X3:1]:[#7X2,#7X3+1:2]                                         :        2 matches
2023-02-21 16:03:10.277 DEBUG: [#6X3:1]=[#7X2,#7X3+1:2]                                         :        0 matches
2023-02-21 16:03:10.278 DEBUG: [#6:1]-[#8:2]                                                    :        0 matches
2023-02-21 16:03:10.279 DEBUG: [#6X4:1]-[#8X2H0:2]                                              :        0 matches
2023-02-21 16:03:10.281 DEBUG: [#6X3:1]-[#8X2:2]                                

2023-02-21 16:03:10.364 DEBUG: [#15:1]-[#35:2]                                                  :        0 matches
2023-02-21 16:03:10.366 DEBUG: [#15:1]-[#53:2]                                                  :        0 matches
2023-02-21 16:03:10.367 DEBUG: [#6X4:1]-[#1:2]                                                  :        3 matches
2023-02-21 16:03:10.368 DEBUG: [#6X3:1]-[#1:2]                                                  :        6 matches
2023-02-21 16:03:10.370 DEBUG: [#6X2:1]-[#1:2]                                                  :        0 matches
2023-02-21 16:03:10.371 DEBUG: [#7:1]-[#1:2]                                                    :        2 matches
2023-02-21 16:03:10.372 DEBUG: [#8:1]-[#1:2]                                                    :        0 matches
2023-02-21 16:03:10.372 DEBUG: 33 matches identified
2023-02-21 16:03:10.374 INFO: 33 bonds added (0 skipped due to constraints)
2023-02-21 16:03:10.374 DEBUG: Finding matches for AngleHandler
20

In [9]:
# Once the mixed system is created, we can run several different types of simulation:

mixedSystem.run_mixed_md(
    steps=1000, interval=100, output_file="./output_md_mixed.pdb"
)

2023-02-21 16:03:43.885 DEBUG: Running mixed MD for 1000 steps
2023-02-21 16:03:46.248 INFO: Minimising energy...


[W parser.cpp:3513] Warning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (function operator())
[W graph_fuser.cpp:104] Warning: operator() profile_node %512 : int[] = prim::profile_ivalue(%510)
 does not have profile information (function operator())


#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-23510.154134881916,29.674656771670332,0
200,0.20000000000000015,-23220.064996524918,55.552738195847915,4.74
300,0.3000000000000002,-22942.921073639285,80.90399159255618,4.72
400,0.4000000000000003,-22709.793857067954,106.90942188340597,4.71
500,0.5000000000000003,-22435.338342838135,120.08971459095267,4.72
600,0.6000000000000004,-22167.606046909263,132.98124201639737,4.72
700,0.7000000000000005,-22052.135877533394,154.11273272664903,4.72
800,0.8000000000000006,-21960.194330289305,168.6604538225054,4.72
900,0.9000000000000007,-21711.367642907237,174.62032204603952,4.72
1000,1.0000000000000007,-21601.381078675608,190.28340465171027,4.73


In [ ]:
# runs Markov chain monte carlo replica exchange to interpoolate between MM and MM/ML descriptions of the system
mixedSystem.run_repex(replicas=3, restart=False, steps=10, equilibration_protocol="gentle")